In [1]:
from sklearn.datasets import make_classification

data = make_classification(n_samples=12500, n_informative=5, n_classes=3, random_state=0)

# test = make_classification(n_samples=10000, n_informative=5, n_classes=3, random_state=42)



In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split

df_data = pd.DataFrame(data[0], columns=[f"f{n}" for n in range(data[0].shape[1])])
df_data['target'] = data[1]
X_train, X_test = train_test_split(df_data, test_size=0.2, random_state=0)

# train = pd.DataFrame(X_train, columns=[f"f{n}" for n in range(data[0].shape[1])])
# train['target'] = y_train
# test = pd.DataFrame(X_test, columns=[f"f{n}" for n in range(data[0].shape[1])])
# test['target'] = y_test

In [7]:
from pycaret.classification import *

s = setup(X_train, target='target', preprocess=False, test_data=X_test)
best = compare_models()
best

,Description,Value
0,Session id,1626
1,Target,target
2,Target type,Multiclass
3,Original data shape,"(12500, 21)"
4,Transformed data shape,"(12500, 21)"
5,Transformed train set shape,"(10000, 21)"
6,Transformed test set shape,"(2500, 21)"
7,Numeric features,20


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
lightgbm,Light Gradient Boosting Machine,0.8874,0.9746,0.8874,0.8877,0.8874,0.8311,0.8313,0.7450
rf,Random Forest Classifier,0.8844,0.9718,0.8844,0.8847,0.8843,0.8266,0.8268,0.3560
et,Extra Trees Classifier,0.8810,0.9715,0.8810,0.8811,0.8808,0.8215,0.8217,0.1360
gbc,Gradient Boosting Classifier,0.8540,0.0000,0.8540,0.8539,0.8538,0.7810,0.7811,2.6450
dt,Decision Tree Classifier,0.8181,0.8636,0.8181,0.8182,0.8180,0.7271,0.7273,0.0350
knn,K Neighbors Classifier,0.8062,0.9271,0.8062,0.8085,0.8065,0.7093,0.7101,0.1170
qda,Quadratic Discriminant Analysis,0.7811,0.0000,0.7811,0.7913,0.7803,0.6715,0.6764,0.0070
lr,Logistic Regression,0.7589,0.0000,0.7589,0.7588,0.7584,0.6383,0.6387,0.2040
lda,Linear Discriminant Analysis,0.7543,0.0000,0.7543,0.7537,0.7528,0.6315,0.6325,0.0070
ridge,Ridge Classifier,0.7512,0.0000,0.7512,0.7500,0.7482,0.6267,0.6287,0.0060


LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               importance_type='split', learning_rate=0.1, max_depth=-1,
               min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
               n_estimators=100, n_jobs=-1, num_leaves=31, objective=None,
               random_state=1626, reg_alpha=0.0, reg_lambda=0.0, subsample=1.0,
               subsample_for_bin=200000, subsample_freq=0)

In [8]:
from sklearn.metrics import accuracy_score

accuracy_score(best.predict(test.drop("target", axis=1)), test['target'])

0.8904

In [19]:
import mlflow

mlflow.set_tracking_uri("http://localhost:5000/")

In [5]:
exp = mlflow.get_experiment_by_name("clf1")

runs = mlflow.search_runs(experiment_names=['clf1'], filter_string='tags.Source = "finalize_model"', order_by=['end_time DESC'], max_results=1)
runs

,run_id,experiment_id,status,artifact_uri,start_time,end_time,metrics.TT,params.n_jobs,params.boosting_type,params.class_weight,...,tags.mlflow.log-model.history,tags.Source,tags.Run ID,tags.mlflow.runName,tags.mlflow.parentRunId,tags.mlflow.user,tags.URI,tags.Run Time,tags.mlflow.source.name,tags.USI
0,3ea36b6027f241c1a742184217ea4aa5,268429473033530207,FINISHED,mlflow-artifacts:/268429473033530207/3ea36b602...,2026-01-03 15:20:46.265000+00:00,2026-01-03 15:20:47.051000+00:00,0.58,-1,gbdt,None,...,"[{""run_id"": ""3ea36b6027f241c1a742184217ea4aa5""...",finalize_model,3ea36b6027f241c1a742184217ea4aa5,Light Gradient Boosting Machine,0853b38b67494cb9b3675a9641d157c2,egor,75f0addb,0.65,./src/train_automl.py,c636


In [4]:
run_id = runs.loc[0, "run_id"]

ver = mlflow.register_model(f"runs:/{run_id}/model", "prod_model", tags={"stage": 'staging'})

Successfully registered model 'prod_model'.
2026/01/03 20:32:11 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: prod_model, version 1
Created version '1' of model 'prod_model'.


In [12]:
ver.last_updated_timestamp

1767454331258

In [17]:
mlflow.search_model_versions(filter_string='tags.stage = "staging"', order_by=['last_updated_timestamp DESC'], max_results=1)

[<ModelVersion: aliases=[], creation_timestamp=1767454681287, current_stage='None', description='', last_updated_timestamp=1767454681287, name='prod_model', run_id='8002d71da8c342b2b36d6006cf632362', run_link='', source='mlflow-artifacts:/268429473033530207/8002d71da8c342b2b36d6006cf632362/artifacts/model', status='READY', status_message='', tags={'stage': 'staging'}, user_id='', version='2'>]

In [18]:
mlflow.search_model_versions(filter_string='tags.stage = "prod"', order_by=['last_updated_timestamp DESC'], max_results=1)

[]

In [20]:
mlflow.search_registered_models()

[]

In [ ]:
vers = mlflow.search_model_versions()

prod = max([ver for ver in vers if ver.tags['stage'] == 'prod'], key=lambda x: x.last_updated_timestamp)

for ver in mlflow.search_model_versions():
    print(ver.tags['stage'])

staging
staging


In [ ]:
exp = mlflow.get_experiment_by_name("clf1")

runs = mlflow.search_runs(experiment_ids=exp.experiment_id, filter_string='tags.Source = "finalize_model"', order_by=['end_time DESC'], max_results=1)

from mlflow import MlflowClient

client = MlflowClient("http://localhost:5000")


# client.create_registered_model("model_prod")
client.create_model_version("model_prod", source=runs.loc[0, "artifact_uri"], run_id=runs.loc[0, "run_id"], tags={"stage": 'staging'})


2026/01/02 17:31:35 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: model_prod, version 2


<ModelVersion: aliases=[], creation_timestamp=1767357095774, current_stage='None', deployment_job_state=<ModelVersionDeploymentJobState: current_task_name='', job_id='', job_state='DEPLOYMENT_JOB_CONNECTION_STATE_UNSPECIFIED', run_id='', run_state='DEPLOYMENT_JOB_RUN_STATE_UNSPECIFIED'>, description='', last_updated_timestamp=1767357095774, metrics=None, model_id=None, name='model_prod', params=None, run_id='e3563079331244b6b6ecbca40679c5c7', run_link='', source='mlflow-artifacts:/188641710580540804/e3563079331244b6b6ecbca40679c5c7/artifacts', status='READY', status_message=None, tags={'stage': 'staging'}, user_id='', version='2'>

In [ ]:
client.model()

In [25]:
runs = mlflow.search_runs(experiment_ids=exp.experiment_id, filter_string='tags.Source = "finalize_model"', order_by=['end_time DESC'], max_results=1)
runs.loc[0, "run_id"]

'e3563079331244b6b6ecbca40679c5c7'

In [ ]:
def pick_best_run(experiment_name: str, metric: str | None, ascending: bool):
    exp = mlflow.get_experiment_by_name(experiment_name)
    if exp is None:
        raise ValueError(f"Эксперимент '{experiment_name}' не найден")

    order_by = []
    if metric:
        direction = "ASC" if ascending else "DESC"
        order_by.append(f"metrics.{metric} {direction}")
    else:
        # если метрика не указана — берём последний по времени
        order_by.append("attributes.start_time DESC")

    runs = mlflow.search_runs(
        experiment_ids=[exp.experiment_id],
        order_by=order_by,
        max_results=1,
    )
    if runs.empty:
        raise ValueError("В эксперименте нет run'ов")

    return runs.iloc[0]["run_id"]
